<a href="https://colab.research.google.com/github/sasatriap/Lampiran-Skripsi/blob/main/SC_Rasio_90_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import libraries
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

In [ ]:
#-----------------------------------------------------------
# 1. PENGUMPULAN DATA
#-----------------------------------------------------------
print("=== 1. PENGUMPULAN DATA ===")
# Load dataset
file_path = 'C:/Users/Want To Sell/Documents/Kuliah/SKRIPSI/sistem percobaan/cobadata2.xlsx'
data = pd.read_excel(file_path)
print(f"Dataset berhasil dimuat. Jumlah data: {data.shape[0]} baris, {data.shape[1]} kolom")
print("Kolom dalam dataset:", list(data.columns))
print(data.head())

In [ ]:
#-----------------------------------------------------------
# 2. DATA PREPROCESSING
#-----------------------------------------------------------
print("\n=== 2. DATA PREPROCESSING ===")

# Pengecekan missing value
print("\n--- Pengecekan Missing Value ---")
missing_summary = data.isnull().sum()
print(missing_summary)

# Menghapus baris dengan missing value jika ada
if missing_summary.any():
    print("\nData memiliki missing value. Menghapus baris dengan nilai kosong...")
    data = data.dropna()
    print(f"Missing value telah dihapus. Data sekarang: {data.shape[0]} baris")
else:
    print("\nData tidak memiliki missing value.")

# Menampilkan data sebelum encoding
print("\n--- Data Sebelum Label Encoding ---")
print(data.head())

# Inisialisasi LabelEncoder
le = LabelEncoder()

# Daftar kolom kategorikal
categorical_columns = ['usia', 'pendidikan', 'jenis_kelamin', 'pekerjaan',
                      'alasan_mengunjungi', 'kedatangan_kembali']

# Dictionary untuk menyimpan hasil mapping encoding
label_mappings = {}

# Proses Label Encoding
for col in categorical_columns:
    data[col] = le.fit_transform(data[col])
    # Simpan mapping label
    label_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))

print("\n--- Data Setelah Label Encoding ---")
print(data.head())

# Menampilkan hasil mapping label encoding untuk tiap kolom
print("\n=== Mapping Label Encoding ===")
for col, mapping in label_mappings.items():
    print(f"\nMapping untuk '{col}':")
    for category, encoded_value in mapping.items():
        print(f"  {category} -> {encoded_value}")

In [ ]:
# Memisahkan fitur dan target
X = data.iloc[:, :-1]  # Semua kolom kecuali kolom terakhir
y = data.iloc[:, -1]   # Kolom terakhir sebagai target

In [ ]:
#-----------------------------------------------------------
# 3. SMOTE (SYNTHETIC MINORITY OVERSAMPLING TECHNIQUE)
#-----------------------------------------------------------
print("\n=== 3. SMOTE ===")
print("Distribusi kelas sebelum SMOTE:")
print(y.value_counts())

# Mendapatkan jumlah total data sebelum SMOTE
total_before = len(y)
class_counts_before = y.value_counts()
percent_before = (class_counts_before / total_before) * 100

# Visualisasi distribusi kelas sebelum SMOTE
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.barplot(x=class_counts_before.index, y=class_counts_before.values, hue=class_counts_before.index,
            palette="coolwarm", legend=False)
plt.title("Distribusi Kelas Sebelum SMOTE", fontsize=14, fontweight="bold", color="darkred")
plt.xlabel("Kelas", fontsize=12, fontweight="bold", color="black")
plt.ylabel("Jumlah Sampel", fontsize=12, fontweight="bold", color="black")
plt.xticks(ticks=[0, 1, 2], labels=["Iya (0)", "Netral (1)", "Tidak (2)"], fontsize=11, color="black")

# Menambahkan label presentase
for i, value in enumerate(class_counts_before.values):
    plt.text(i, value + 5, f"{percent_before.iloc[i]:.2f}%", ha='center', fontsize=10, fontweight="bold", color="black")

# Menerapkan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("\nDistribusi kelas setelah SMOTE:")
print(y_resampled.value_counts())

# Mendapatkan jumlah total data setelah SMOTE
total_after = len(y_resampled)
class_counts_after = y_resampled.value_counts()
percent_after = (class_counts_after / total_after) * 100

# Visualisasi distribusi kelas setelah SMOTE
plt.subplot(1, 2, 2)
sns.barplot(x=class_counts_after.index, y=class_counts_after.values, hue=class_counts_after.index,
            palette="coolwarm", legend=False)
plt.title("Distribusi Kelas Setelah SMOTE", fontsize=14, fontweight="bold", color="darkgreen")
plt.xlabel("Kelas", fontsize=12, fontweight="bold", color="black")
plt.ylabel("Jumlah Sampel", fontsize=12, fontweight="bold", color="black")
plt.xticks(ticks=[0, 1, 2], labels=["Iya (0)", "Netral (1)", "Tidak (2)"], fontsize=11, color="black")

# Menambahkan label presentase
for i, value in enumerate(class_counts_after.values):
    plt.text(i, value + 5, f"{percent_after.iloc[i]:.2f}%", ha='center', fontsize=10, fontweight="bold", color="black")

plt.tight_layout()
plt.show()

In [ ]:
#-----------------------------------------------------------
# 4. SELEKSI FITUR CHI-SQUARE
#-----------------------------------------------------------
print("\n=== 4. SELEKSI FITUR CHI-SQUARE ===")

# Menghitung Chi-Square Score
print("Perhitungan Chi-Square Score")
chi_scores, _ = chi2(X_resampled, y_resampled)
feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'Chi-Square Score': chi_scores
})

# Mengurutkan fitur berdasarkan Chi-Square Score
print("\nChi-Square Score untuk setiap fitur:")
feature_scores_sorted = feature_scores.sort_values(by='Chi-Square Score', ascending=False)
print("Fitur diurutkan berdasarkan Chi-Square Score (dari tertinggi):")
print(feature_scores_sorted)

# Penerapan Cutoff Persentase pada Semua Fitur
print("\n--- Penerapan Cutoff Persentase pada Fitur ---")
cutoff_percentages = [0.25, 0.50, 0.75, 1.0]
feature_sets = {}  # Untuk menyimpan fitur-fitur terpilih untuk setiap cutoff

for cutoff in cutoff_percentages:
    n_features = max(1, int(len(X.columns) * cutoff))  # Minimal 1 fitur
    selected_features = feature_scores_sorted['Feature'].head(n_features).tolist()

    feature_sets[f"top_{int(cutoff * 100)}pct"] = selected_features

    print(f"\nCutoff {cutoff*100}% menghasilkan {n_features} fitur:")
    for i, feature in enumerate(selected_features, 1):
        chi_score = feature_scores_sorted.loc[
            feature_scores_sorted['Feature'] == feature, 'Chi-Square Score'
        ].values[0]
        print(f"{i}. {feature} (Chi-Square: {chi_score:.4f})")

# 4.4 Visualisasi Hasil Seleksi Fitur
print("\n--- Visualisasi Hasil Seleksi Fitur ---")
# Visualisasi Chi-Square Scores untuk semua fitur
plt.figure(figsize=(12, 6))
sns.barplot(x='Feature', y='Chi-Square Score', data=feature_scores_sorted)
plt.title('Chi-Square Scores of All Features')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
#-----------------------------------------------------------
# 5. SPLITTING DATA
#-----------------------------------------------------------
print("\n=== 5. SPLITTING DATA ===")
# Membagi dataset menjadi data training dan testing
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.1, random_state=42
)

print(f"Jumlah data training: {X_train.shape[0]} sampel")
print(f"Jumlah data testing: {X_test.shape[0]} sampel")
print(f"Rasio pembagian: {X_train.shape[0] / (X_train.shape[0] + X_test.shape[0]):.2f} : {X_test.shape[0] / (X_train.shape[0] + X_test.shape[0]):.2f}")

# Menampilkan distribusi kelas pada data training
train_counts = np.bincount(y_train)
print("\nDistribusi kelas pada data training:")
print(f"Iya (0): {train_counts[0]} sampel ({train_counts[0]/len(y_train):.2f})")
print(f"Netral (1): {train_counts[1]} sampel ({train_counts[1]/len(y_train):.2f})")
print(f"Tidak (2): {train_counts[2]} sampel ({train_counts[2]/len(y_train):.2f})")

# Menampilkan distribusi kelas pada data testing
test_counts = np.bincount(y_test)
print("\nDistribusi kelas pada data testing:")
print(f"Iya (0): {test_counts[0]} sampel ({test_counts[0]/len(y_test):.2f})")
print(f"Netral (1): {test_counts[1]} sampel ({test_counts[1]/len(y_test):.2f})")
print(f"Tidak (2): {test_counts[2]} sampel ({test_counts[2]/len(y_test):.2f})")

In [ ]:
#-----------------------------------------------------------
# 6. KLASIFIKASI
#-----------------------------------------------------------
print("=== 6. KLASIFIKASI KNN, RANDOM FOREST, XGBOOST ===")

# X_train_features: Fitur (independen) dari data pelatihan
# X_test_features: Fitur dari data pengujian
# y_train: Label/target dari data pelatihan
# y_test: Label/target dari data pengujian
# feature_set_name: Nama atau label dari set fitur yang digunakan (misalnya: 'top_50pct')

# Fungsi evaluate model digunakan untuk menerima input
def evaluate_model(X_train_features, X_test_features, y_train, y_test, feature_set_name):

    # Inisialisasi tiga model klasifikasi: K-NN, Random Forest, dan XGBoost
    models = {
        'K-NN': KNeighborsClassifier(),
        'Random Forest': RandomForestClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42, eval_metric='mlogloss')
    }

    # results: Dictionary untuk menyimpan hasil klasifikasi tiap model
    results = {}

    # Melakukan pelatihan dan evaluasi pada setiap model
    for name, model in models.items():
        # Melatih model menggunakan data training
        model.fit(X_train_features, y_train)

        # Melakukan prediksi terhadap data testing
        y_pred = model.predict(X_test_features)

        # accuracy: Nilai akurasi dari model (dalam persen)
        accuracy = accuracy_score(y_test, y_pred) * 100

        # conf_matrix: Matriks konfusi dari hasil prediksi
        conf_matrix = confusion_matrix(y_test, y_pred)

        # class_counts: Jumlah prediksi per kelas dari hasil y_pred
        class_counts = Counter(y_pred)
        sorted_counts = {key: class_counts[key] for key in sorted(class_counts)}

        # Menyimpan semua hasil ke dalam dictionary untuk model saat ini
        results[name] = {
            'accuracy': accuracy,
            'predictions': y_pred,
            'conf_matrix': conf_matrix
        }

        # Menampilkan informasi hasil evaluasi di terminal
        print(f"\nModel: {name}")
        print(f"Akurasi: {accuracy:.2f}%")
        print(f"Jumlah prediksi per kelas (0, 1, 2): {sorted_counts}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        # Visualisasi confusion matrix menggunakan heatmap
        plt.figure(figsize=(6, 5))
        cmap_choice = "Blues" if name == "K-NN" else "Greens" if name == "Random Forest" else "Reds"
        sns.heatmap(
            conf_matrix, annot=True, fmt='d', cmap=cmap_choice,
            xticklabels=['Iya', 'Netral', 'Tidak'], yticklabels=['Iya', 'Netral', 'Tidak']
        )
        plt.title(f"Confusion Matrix - {name} ({feature_set_name})")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()

    # Mengembalikan hasil evaluasi seluruh model
    return results

#-----------------------------------------------------------
# Menjalankan evaluasi untuk setiap set fitur yang telah dipilih sebelumnya
#-----------------------------------------------------------

# all_results: Dictionary untuk menyimpan hasil klasifikasi dari tiap kombinasi fitur dan model
all_results = {}

# Melakukan iterasi untuk setiap kombinasi fitur berdasarkan nama cut-off (misalnya: 25%, 50%, 75%, 100%)
for cutoff_name, selected_features in feature_sets.items():
    # Memilih fitur tertentu dari data training dan testing sesuai dengan cut-off saat ini
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Memanggil fungsi evaluate model untuk set fitur ini dan menyimpan dalam eval_results
    all_results[cutoff_name] = evaluate_model(
        X_train_selected, X_test_selected, y_train, y_test, cutoff_name
    )

#-----------------------------------------------------------
# Menentukan model terbaik dari seluruh hasil klasifikasi
#-----------------------------------------------------------

# best_model: Menyimpan nama algoritma terbaik (berdasarkan akurasi tertinggi)
# best_accuracy: Menyimpan nilai akurasi tertinggi yang ditemukan
# best_cutoff: Menyimpan nama set fitur (cut-off presentase) yang menghasilkan model terbaik
best_model = None
best_accuracy = 0
best_cutoff = None

# Loop untuk mencari model dengan akurasi tertinggi dari seluruh kombinasi
for cutoff_name, results in all_results.items():
    for model_name, model_results in results.items():
        if model_results['accuracy'] > best_accuracy:
            best_accuracy = model_results['accuracy']
            best_model = model_name
            best_cutoff = cutoff_name

# Menampilkan hasil akhir berupa model terbaik dengan akurasi tertinggi
print(f"\nModel terbaik: {best_model} dengan fitur {best_cutoff}, Akurasi: {best_accuracy:.2f}%")

In [ ]:
#-----------------------------------------------------------
# 7. ANALASIS FAKTOR KELOMPOK USIA
#-----------------------------------------------------------
print("\n=== 7.ANALISIS FAKTOR BERDASARKAN KELOMPOK USIA ===")

def analisis_faktor_per_usia(data, encoders):
    # Reverse mapping untuk kategori
    kedatangan_reverse = {v: k for k, v in encoders['kedatangan_kembali'].items()}
    usia_reverse = {v: k for k, v in encoders['usia'].items()}

    # Pilih kolom fitur (kecuali usia & kedatangan_kembali)
    feature_columns = [col for col in data.columns if col not in ['usia', 'kedatangan_kembali']]

    plt.figure(figsize=(12, 10))

    for i, (usia_code, usia_label) in enumerate(usia_reverse.items(), start=1):
        print(f"\n--- Kelompok Usia: {usia_label} ---")

        # Filter data per kelompok usia
        subset = data[data['usia'] == usia_code]
        X, y = subset[feature_columns], subset['kedatangan_kembali']

        print(f"Jumlah sampel: {len(X)}")
        print("Distribusi Kedatangan Kembali:")
        print(y.value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

        # Latih Random Forest
        model = RandomForestClassifier(random_state=42)
        model.fit(X, y)

        # Ambil 10 fitur terpenting
        feat_importance = pd.DataFrame({
            'Feature': feature_columns,
            'Importance': model.feature_importances_
        }).nlargest(10, 'Importance')

        print("\nTop 10 Faktor yang Mempengaruhi Kedatangan Kembali:")
        print(feat_importance.to_string(index=False))

        # Plot hasil
        plt.subplot(3, 1, i)
        sns.barplot(x='Importance', y='Feature', data=feat_importance, palette='viridis')
        plt.title(f'Faktor Penting - Usia: {usia_label}', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.suptitle('Faktor yang Mempengaruhi Kedatangan Kembali Berdasarkan Usia',
                 fontsize=14, fontweight='bold', y=1.02)
    plt.show()

# Panggil fungsi analisis
analisis_faktor_per_usia(data, label_mappings)
